# DANIEL CORRAL RUIZ

# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

Comenzamos cargando el contexto spark y añadiendo los datasets.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz #d
!pip install -q findspark 

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
viviendas = "BDpracticafinalCSV.csv" # subimos el archivo
viviendas = sc.textFile(viviendas)
primera_fila = viviendas.take(1)[0] 
viviendas = viviendas.filter(lambda x: x != primera_fila) #Nos quedamos con todo lo que no sea la primera fila
viviendas = viviendas.map(lambda x: x.split(";")) # separamos por comas
viviendas.take(1)


[['1',
  'RL',
  '141',
  '31770',
  'Lvl',
  'Gtl',
  '6',
  '1960',
  '1960',
  'Stone',
  'TA',
  'Gd',
  '1080',
  'Fa',
  'Y',
  '1',
  '0',
  '3',
  '1',
  'TA',
  '7',
  '2',
  '528',
  'TA',
  '0',
  'Sp',
  '5',
  '2010',
  '215000']]

Importamos los paquetes de python numpy y pandas para una mayor facilidad a la hora de tratar con los datos.

In [16]:
import pandas as pd
import numpy as np

In [17]:
viviendaspd = pd.read_csv("BDpracticafinalCSV.csv", sep=";")
viviendaspd

,Order,MS Zoning,Lot Frontage,Lot Area,Land Contour,Land Slope,Overall Qual,Year Built,Year Remod/Add,Mas Vnr Type,Exter Qual,Bsmt Cond,Total Bsmt SF,Heating QC,Central Air,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Garage Cars,Garage Area,Garage Cond,Pool Area,Pool QC,Mo Sold,Yr Sold,SalePrice
0,1,RL,141.0,31770,Lvl,Gtl,6,1960,1960,Stone,TA,Gd,1080,Fa,Y,1,0,3,1,TA,7,2.0,528.0,TA,0,Sp,5,2010,215000
1,2,RH,80.0,11622,Lvl,Gtl,5,1961,1961,None,TA,TA,882,TA,Y,1,0,2,1,TA,5,1.0,730.0,TA,0,Sp,6,2010,105000
2,3,RL,81.0,14267,Lvl,Gtl,6,1958,1958,BrkFace,TA,TA,1329,TA,Y,1,1,3,1,Gd,6,1.0,312.0,TA,0,Sp,6,2010,172000
3,4,RL,93.0,11160,Lvl,Gtl,7,1968,1968,None,Gd,TA,2110,Ex,Y,2,1,3,1,Ex,8,2.0,522.0,TA,0,Sp,4,2010,244000
4,5,RL,74.0,13830,Lvl,Gtl,5,1997,1998,None,TA,TA,928,Gd,Y,2,1,3,1,TA,6,2.0,482.0,TA,0,Sp,3,2010,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2930,RL,74.0,9627,Lvl,Mod,7,1993,1994,BrkFace,TA,TA,996,Ex,Y,2,1,3,1,TA,9,3.0,650.0,TA,0,Sp,11,2006,188000
2932,2898,Rl,NaN,11170,Lvl,Gtl,7,1990,1991,None,TA,TA,1216,Ex,Y,2,1,4,1,TA,8,2.0,693.0,TA,0,Sp,4,2006,250000
2933,2899,Rl,NaN,8098,Lvl,Gtl,6,2000,2000,None,Gd,TA,1381,Ex,Y,2,0,2,1,Gd,5,2.0,470.0,TA,0,Sp,10,2006,202000
2934,2900,Rl,85.0,14331,Lvl,Gtl,8,2002,2002,BrkFace,Gd,TA,1800,Ex,Y,2,0,3,1,Gd,7,3.0,765.0,TA,0,Sp,5,2006,312500


Hemos leído el segundo archivos de datos unicamente la segunda hoja de nuestro dataset, el cual nos proporciona información acerca de los datos.

In [52]:
viviendaspd_1 = pd.read_excel('BDpracticafinal2021.xlsx', sheet_name='Diseño de registro')
viviendaspd_1

,NOMBRE VARIABLE,DESCRIPTOR,VALORES
0,Order,Variable de identificación,1 a 2930
1,MS Zoning,Zona de ubicación de la vivienda,A rural\n C comercial\n FV ...
2,Lot Frontage,Longitud de la fachada en pies,NaN
3,Lot Area,Superficie de la vivienda en pies cuadrados,NaN
4,Land Contour,Contorno del terreno circundante,Lvl llano \n Bnk Tipo bancal\n ...
5,Land Slope,Tipo de pendiente de la vivienda,Gtl pendiente suave\n Mod pendien...
6,Overall Qual,Grado de calidad de materiales y acabado de la...,De 1 (Muy pobre) a 10 (Excelente)
7,Year Built,Año de construccion de la vivienda,NaN
8,Year Remod/Add,Año de última reforma de la vivienda,NaN
9,Mas Vnr Type,Tipo de revestimiento exterior,BrkCmn Ladrillo normal\n BrkFace ...


Comprobamos los distintos valores que tenemos en la variable 'MS Zoning'

In [18]:
viviendaspd['MS Zoning'].unique()

array(['RL', 'RH', 'FV', 'RM', 'C', 'I', 'A', 'Rl', 'rL'], dtype=object)

Y comprobamos el tipo de datos:

In [6]:
viviendaspd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order           2936 non-null   int64  
 1   MS Zoning       2936 non-null   object 
 2   Lot Frontage    2444 non-null   float64
 3   Lot Area        2936 non-null   int64  
 4   Land Contour    2936 non-null   object 
 5   Land Slope      2936 non-null   object 
 6   Overall Qual    2936 non-null   int64  
 7   Year Built      2936 non-null   int64  
 8   Year Remod/Add  2936 non-null   int64  
 9   Mas Vnr Type    2913 non-null   object 
 10  Exter Qual      2936 non-null   object 
 11  Bsmt Cond       2936 non-null   object 
 12  Total Bsmt SF   2936 non-null   int64  
 13  Heating QC      2936 non-null   object 
 14  Central Air     2936 non-null   object 
 15  Full Bath       2936 non-null   int64  
 16  Half Bath       2936 non-null   int64  
 17  Bedroom AbvGr   2936 non-null   i

Viviendas ubicadas en zonas residenciales (alta, media y baja densidad).

In [19]:
filtro1 = viviendaspd['MS Zoning'] == 'RH'
filtro2 = viviendaspd['MS Zoning'] == 'RL'
filtro3 = viviendaspd['MS Zoning'] == 'RM'
filtro4 = viviendaspd['MS Zoning'] == 'Rl'
filtro5 = viviendaspd['MS Zoning'] == 'rL'

viviendaspd = viviendaspd[filtro1 | filtro2 | filtro3 | filtro4 | filtro5]
viviendaspd

,Order,MS Zoning,Lot Frontage,Lot Area,Land Contour,Land Slope,Overall Qual,Year Built,Year Remod/Add,Mas Vnr Type,Exter Qual,Bsmt Cond,Total Bsmt SF,Heating QC,Central Air,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Garage Cars,Garage Area,Garage Cond,Pool Area,Pool QC,Mo Sold,Yr Sold,SalePrice
0,1,RL,141.0,31770,Lvl,Gtl,6,1960,1960,Stone,TA,Gd,1080,Fa,Y,1,0,3,1,TA,7,2.0,528.0,TA,0,Sp,5,2010,215000
1,2,RH,80.0,11622,Lvl,Gtl,5,1961,1961,None,TA,TA,882,TA,Y,1,0,2,1,TA,5,1.0,730.0,TA,0,Sp,6,2010,105000
2,3,RL,81.0,14267,Lvl,Gtl,6,1958,1958,BrkFace,TA,TA,1329,TA,Y,1,1,3,1,Gd,6,1.0,312.0,TA,0,Sp,6,2010,172000
3,4,RL,93.0,11160,Lvl,Gtl,7,1968,1968,None,Gd,TA,2110,Ex,Y,2,1,3,1,Ex,8,2.0,522.0,TA,0,Sp,4,2010,244000
4,5,RL,74.0,13830,Lvl,Gtl,5,1997,1998,None,TA,TA,928,Gd,Y,2,1,3,1,TA,6,2.0,482.0,TA,0,Sp,3,2010,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2930,RL,74.0,9627,Lvl,Mod,7,1993,1994,BrkFace,TA,TA,996,Ex,Y,2,1,3,1,TA,9,3.0,650.0,TA,0,Sp,11,2006,188000
2932,2898,Rl,NaN,11170,Lvl,Gtl,7,1990,1991,None,TA,TA,1216,Ex,Y,2,1,4,1,TA,8,2.0,693.0,TA,0,Sp,4,2006,250000
2933,2899,Rl,NaN,8098,Lvl,Gtl,6,2000,2000,None,Gd,TA,1381,Ex,Y,2,0,2,1,Gd,5,2.0,470.0,TA,0,Sp,10,2006,202000
2934,2900,Rl,85.0,14331,Lvl,Gtl,8,2002,2002,BrkFace,Gd,TA,1800,Ex,Y,2,0,3,1,Gd,7,3.0,765.0,TA,0,Sp,5,2006,312500


Viviendas ubicadas en zonas residenciales (alta, media y baja densidad).

In [20]:
RL = viviendas.filter(lambda x: x[1] == 'RL')
RH = viviendas.filter(lambda x: x[1] == 'RH')
RM = viviendas.filter(lambda x: x[1] == 'RM')
rL = viviendas.filter(lambda x: x[1] == 'rL')
Rl = viviendas.filter(lambda x: x[1] == 'Rl')
join = RL.union(RH)
join = join.union(RM)
join = join.union(rL)
viv_fin = join.union(Rl)
viv_fin.count()

2768

Observamos como existe un número significativo de nulos en la variable "Lot Frontage"

In [21]:
viviendaspd.isna().sum()

Order               0
MS Zoning           0
Lot Frontage      469
Lot Area            0
Land Contour        0
Land Slope          0
Overall Qual        0
Year Built          0
Year Remod/Add      0
Mas Vnr Type       16
Exter Qual          0
Bsmt Cond           0
Total Bsmt SF       0
Heating QC          0
Central Air         0
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Garage Cars         1
Garage Area         1
Garage Cond         2
Pool Area           0
Pool QC             0
Mo Sold             0
Yr Sold             0
SalePrice           0
dtype: int64

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Hemos comprobado como hay un total de 6 observaciones duplicadas.

In [22]:
viviendaspd.duplicated().sum()

6

Esto puede deberse a que existen casas que se encuentran en la misma urbanización o zona con igual de características.

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

In [23]:
zona_RL = RL.map(lambda x: (x[28]))
zona_RL=zona_RL.filter(lambda x_int: x_int.isdigit())
zona_RL=zona_RL.map(lambda x: float(x))

zona_RL.mean(), zona_RL.count()

(191131.80581241753, 2271)

Observamos el número total de viviendas para cada una de las tres zonas.

In [25]:
viviendaspd.groupby("MS Zoning").Order.count()

MS Zoning
RH      27
RL    2279
RM     462
Name: Order, dtype: int64

Observamos el precio medio para cada una de las tres zonas.

In [26]:
viviendaspd.groupby("MS Zoning").SalePrice.mean()

MS Zoning
RH    136419.777778
RL    191412.606845
RM    126781.393939
Name: SalePrice, dtype: float64

## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Definimos función para calcular las décadas.

In [24]:
def get_decade(x):
    return x[0] + x[1] + x[2] + "0s"

In [62]:
decada = viv_fin.map(lambda x: (get_decade(x[7]), [float(x[3])]))
decada.reduceByKey(lambda x,y: [x[0]+y[0]]).take(15)

[('1900s', [343871.0]),
 ('1980s', [1183432.0]),
 ('1890s', [132274.0]),
 ('1970s', [3349636.0]),
 ('1910s', [955525.0]),
 ('1960s', [4346102.0]),
 ('1870s', [25092.0]),
 ('2010s', [18754.0]),
 ('1920s', [1544611.0]),
 ('1950s', [3884896.0]),
 ('1880s', [115197.0]),
 ('1990s', [3458945.0]),
 ('1930s', [867047.0]),
 ('1940s', [1397747.0]),
 ('2000s', [6934543.0])]

In [64]:
ex = viv_fin.filter(lambda x: (x[13]== "Ex"))
ex = ex.map(lambda x: (get_decade(x[7]), x[13]))
ex.countByKey()

defaultdict(int,
            {'1880s': 2,
             '1890s': 4,
             '1900s': 12,
             '1910s': 38,
             '1920s': 51,
             '1930s': 40,
             '1940s': 56,
             '1950s': 98,
             '1960s': 88,
             '1970s': 68,
             '1980s': 28,
             '1990s': 235,
             '2000s': 641,
             '2010s': 2})

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

In [65]:
año_2009 = viv_fin.filter(lambda x: x[27] == '2009')

Casas_Caras = año_2009.map(lambda x: (x[27], [float(x[28]), float(x[3]) * 0.09290304]))

Casas_Caras.map(lambda x: [x[0],x[1][0]/x[1][1]]).top(10)

[['2009', 1026.7114551323118],
 ['2009', 991.0320746567418],
 ['2009', 821.056658770186],
 ['2009', 807.9610176314369],
 ['2009', 765.6471992838166],
 ['2009', 756.0365649831829],
 ['2009', 717.5940277806482],
 ['2009', 714.2814183044536],
 ['2009', 711.1869382468924],
 ['2009', 701.0704810883306]]

## Media anual por zonas del precio de venta y metros cuadrados.

In [66]:
RM_precio = RM.map(lambda x: (x[27], [float(x[28]) / len(x[28]), 1]))
RM_reduced = RM_precio.reduceByKey(lambda x,y: [x[0] + y[0]])
RM_metros = RM.map(lambda x: (x[27], [(float(x[3]) * 0.09) / len(x[3])]))
RM_metros.reduceByKey(lambda x,y: [x[0] + y[0]]).take(10)

[('2010', [7928.072999999998]),
 ('2007', [13706.540999999997]),
 ('2009', [10804.941]),
 ('2008', [17486.181000000004]),
 ('2006', [13806.2925])]

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

Realizamos filtros para cada uno de los tipos de revistimientos y observamos una clara diferencia de recaudación en aquellas casas que son de ladrillo.

In [68]:
piedra = viv_fin.filter(lambda x: x[9] == 'Stone')
ladrillo = viv_fin.filter(lambda x: x[9] == 'BrkFace' or 'BrkCmn')
piedra = piedra.map(lambda x: (x[28]))
piedra=piedra.filter(lambda x_int: x_int.isdigit())
piedra=piedra.map(lambda x: int(x))
ladrillo = ladrillo.map(lambda x: (x[28]))
ladrillo=ladrillo.filter(lambda x_int: x_int.isdigit())
ladrillo=ladrillo.map(lambda x: int(x))
piedra.sum(), ladrillo.sum()

(60495477, 498485669)

In [69]:
piedra.sum() - ladrillo.sum()

-437990192

Observamos una diferencia de recaudación de 437990192 entre ambos tipos.


## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

In [71]:
dosomas = viv_fin.filter(lambda x: x[18] == '2' and x [21] == '2' or '3' or '4' or '5')
dosomas = dosomas.map(lambda x: (x[28]))
dosomas=dosomas.filter(lambda x_int: x_int.isdigit())
dosomas=dosomas.map(lambda x: int(x))

In [72]:
uno = viv_fin.filter(lambda x: x[18] == '1' and x [21] == '1')
uno = uno.map(lambda x: (x[28]))
uno=uno.filter(lambda x_int: x_int.isdigit())
uno=uno.map(lambda x: int(x))

In [73]:
dosomas.mean(), uno.mean()

(180088.75325144516, 128121.99337748351)

In [40]:
dosomas.stdev(), uno.stdev()

(80213.95568776174, 30283.73797627781)

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

Comprobamos la media de los precios de ventas agrupados por el número de vehículos que entran en el garaje. En nuestro caso observamos un precio mayor medio en aquellas viviendas con disponibilidad de 3 vehículos.

In [61]:
viviendaspd.groupby('Garage Cars')['SalePrice'].mean()

Garage Cars
0.0    107023.639456
1.0    128011.655759
2.0    181890.093180
3.0    311831.488827
4.0    228748.687500
5.0    126500.000000
Name: SalePrice, dtype: float64

## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta